# 0. Import Dataset

In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv(r"C:\Users\alexs\Documents\TFM_MBD\TFM_MBD_2024_AlexSerra\preprocessed_data\other_europe.csv")

# 1. Defining train, validation, test

In [3]:
train_lim = int(df.shape[0]*0.7)
df_train = df.iloc[:train_lim]
df_test = df.iloc[train_lim:]

In [4]:
train_lim_meta = int(df_train.shape[0]*0.6)
df_train_meta = df_train.iloc[:train_lim_meta]
df_validation_meta = df_train.iloc[train_lim_meta:]

Let's define general datasets:

- df_train --> data that we are going to use to train our algorithms (prophet, autoarima, ....)
- df_test --> data that only will be used to evaluate scores of how the algorithm work.

Now, we also defined "meta" dataset. This data will be used to train the meta learner:

- df_train_meta --> son algorithm (prophet, autoarima,...) will be trained with this reduced amount of the train dataset.
- df_validation_meta --> this data will be used, after having son algorithms trained, to train the meta learner. For example, for a linear regression, it will be used to obtain the coefficients.

Once we have the metalearner trained, son algorithms will be trained with full df_train. And then using the metalearner to obtain final predictions.

# 2. Prophet

In [5]:
from prophet import Prophet

In [6]:
model_prophet = Prophet()

In [7]:
model_prophet.fit(df_train)

11:18:41 - cmdstanpy - INFO - Chain [1] start processing
11:18:41 - cmdstanpy - INFO - Chain [1] done processing


In [8]:
forecast_prophet = model_prophet.predict(df_test[["ds"]])["yhat"].values

In [9]:
df_final_predictions= pd.DataFrame()

In [10]:
df_final_predictions["ds"] = df_test["ds"]

In [11]:
df_final_predictions ["prophet"] = forecast_prophet

In [12]:
def calculate_mape(actual, predicted):
    actual, predicted = np.array(actual), np.array(predicted)
    return np.mean(np.abs((actual - predicted) / actual)) * 100

In [13]:
mape=calculate_mape(df_test["y"],forecast_prophet)

print(f'MAPE: {mape:.2f}%')

MAPE: 21.35%


# 3. Prophet adding regressors

In [14]:
model_prophet_multiva = Prophet()

In [15]:
for e in df_train.columns[2:]:
    model_prophet_multiva.add_regressor(e)

In [16]:
model_prophet_multiva.fit(df_train)

11:18:41 - cmdstanpy - INFO - Chain [1] start processing
11:18:42 - cmdstanpy - INFO - Chain [1] done processing


In [17]:
forecast_prophet_multiva = model_prophet_multiva.predict(df_test.drop(columns="y"))["yhat"].values

In [18]:
mape=calculate_mape(df_test["y"],forecast_prophet_multiva)

print(f'MAPE: {mape:.2f}%')

MAPE: 21.87%


In [19]:
df_final_predictions["prophet_multiva"] = forecast_prophet_multiva

# 4. Autoarima

In [20]:
from pmdarima import auto_arima

In [21]:
df_train_arima = df_train.copy()
df_test_arima = df_test.copy()

df_train_arima.set_index('ds', inplace=True)
df_train_arima = df_train_arima[["y"]]

df_test_arima.set_index('ds', inplace=True)
df_test_arima = df_test_arima[["y"]]

In [22]:
# Train AutoARIMA model
model_autoarima = auto_arima(df_train_arima, 
                   seasonal=True,  # Change to True if you want to fit a seasonal ARIMA

                   stepwise=True,   # Set to False to perform a more exhaustive search
                   trace=True)      # Set to True to see the search progress

# Print the best model parameters
print(model_autoarima.summary())

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=268.565, Time=0.35 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=325.069, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=278.007, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=292.544, Time=0.05 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=646.754, Time=0.02 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=267.768, Time=0.08 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=290.754, Time=0.05 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=265.856, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=267.825, Time=0.19 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=274.014, Time=0.08 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=272.651, Time=0.05 sec

Best model:  ARIMA(1,0,1)(0,0,0)[0] intercept
Total fit time: 1.081 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  128
Model: 

In [23]:
forecast_autoarima = model_autoarima.predict(n_periods=df_test_arima.shape[0]).values

In [24]:
df_final_predictions["autoarima"] = forecast_autoarima

In [25]:
mape=calculate_mape(df_test["y"],forecast_autoarima)

print(f'MAPE: {mape:.2f}%')

MAPE: 21.07%


# 5. Ensembling (preparing data)

In [26]:
model_prophet_meta = Prophet()

model_prophet_meta.fit(df_train_meta)

forecast_prophet_meta = model_prophet_meta.predict(df_validation_meta[["ds"]])["yhat"].values

df_meta= pd.DataFrame()

df_meta["ds"] = df_validation_meta["ds"]

df_meta ["prophet"] = forecast_prophet_meta

11:18:46 - cmdstanpy - INFO - Chain [1] start processing
11:18:46 - cmdstanpy - INFO - Chain [1] done processing


In [27]:
model_prophet_multiva_meta = Prophet()

for e in df_train_meta.columns[2:]:
    model_prophet_multiva_meta.add_regressor(e)

model_prophet_multiva_meta.fit(df_train_meta)

forecast_prophet_multiva_meta = model_prophet_multiva_meta.predict(df_validation_meta.drop(columns="y"))["yhat"].values

df_meta["prophet_multiva"] = forecast_prophet_multiva_meta

11:18:46 - cmdstanpy - INFO - Chain [1] start processing
11:18:46 - cmdstanpy - INFO - Chain [1] done processing


In [28]:
df_train_arima_meta = df_train_meta.copy()
df_test_arima_meta = df_validation_meta.copy()

df_train_arima_meta.set_index('ds', inplace=True)
df_train_arima_meta = df_train_arima_meta[["y"]]

df_test_arima_meta.set_index('ds', inplace=True)
df_test_arima_meta = df_test_arima_meta[["y"]]

# Train AutoARIMA model
model_autoarima_meta = auto_arima(df_train_arima, 
                   seasonal=True,  # Change to True if you want to fit a seasonal ARIMA

                   stepwise=True,   # Set to False to perform a more exhaustive search
                   trace=True)      # Set to True to see the search progress

# Print the best model parameters
print(model_autoarima_meta.summary())

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept   : AIC=268.565, Time=0.12 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept   : AIC=325.069, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AIC=278.007, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept   : AIC=292.544, Time=0.03 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=646.754, Time=0.01 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept   : AIC=267.768, Time=0.08 sec
 ARIMA(0,0,2)(0,0,0)[0] intercept   : AIC=290.754, Time=0.04 sec
 ARIMA(1,0,1)(0,0,0)[0] intercept   : AIC=265.856, Time=0.06 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept   : AIC=267.825, Time=0.16 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept   : AIC=274.014, Time=0.07 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=272.651, Time=0.05 sec

Best model:  ARIMA(1,0,1)(0,0,0)[0] intercept
Total fit time: 0.719 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  128
Model: 

In [29]:
forecast_autoarima_meta = model_autoarima_meta.predict(n_periods=df_validation_meta.shape[0]).values

df_meta["autoarima"] = forecast_autoarima_meta

In [30]:
df_meta["average_predictions"] = df_meta[['prophet', 'prophet_multiva', 'autoarima']].mean(axis=1)

In [31]:
df_meta["real_values"] = df_validation_meta["y"]

In [32]:
mape=calculate_mape(df_test["y"],forecast_autoarima)

print(f'MAPE: {mape:.2f}%')

MAPE: 21.07%


In [33]:
for e in df_meta.columns[1:-1]:
    mape=calculate_mape(df_meta["real_values"],df_meta[e])
    print(f'MAPE of {e}: {mape:.2f}%')

MAPE of prophet: 27.26%
MAPE of prophet_multiva: 18.22%
MAPE of autoarima: 21.16%
MAPE of average_predictions: 20.45%


# 6. Ensembling (Training Linear Regresion)

In [34]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [35]:
df_meta = df_meta.reset_index(drop=True)

In [36]:
lr = LinearRegression()

In [87]:
lim=int(df_meta.shape[0]*0.7)
X_train = df_meta[["prophet", "prophet_multiva", "autoarima"]].iloc[:lim]
y_train = df_meta [["real_values"]].iloc[:lim]

X_test=df_meta[["prophet", "prophet_multiva", "autoarima"]].iloc[lim:]
y_test = df_meta [["real_values"]].iloc[lim:]

In [89]:
df_meta

,ds,prophet,prophet_multiva,autoarima,average_predictions,real_values
0,2021-06-21,2.519878,2.296073,2.579606,2.465185,2.189091
1,2021-06-28,2.515586,2.241079,2.613922,2.456862,2.053776
2,2021-07-05,2.511294,2.375948,2.644592,2.510611,3.450657
3,2021-07-12,2.507003,2.546921,2.672004,2.575309,2.555231
4,2021-07-19,2.502711,2.155432,2.696504,2.451549,1.966881
5,2021-07-26,2.498419,2.448137,2.718401,2.554986,1.333078
6,2021-08-02,2.494127,2.430990,2.737972,2.554363,2.489611
7,2021-08-09,2.489836,2.352727,2.755464,2.532676,1.828494
8,2021-08-16,2.485544,2.667610,2.771098,2.641417,2.112152
9,2021-08-23,2.481252,2.486315,2.785070,2.584212,2.578539


In [38]:
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)

In [39]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 0.3515840287877502
R^2 Score: 0.23368631676424279


In [40]:
y_pred = lr.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 0.39001008221362243
R^2 Score: 0.38343398507475457


# 7. Final Predictions

In [41]:
df_final_predictions["average_predictions"] = df_final_predictions[['prophet', 'prophet_multiva', 'autoarima']].mean(axis=1)

In [42]:
testing_X = df_final_predictions[['prophet', 'prophet_multiva', 'autoarima']]

In [43]:
# We decide to train the algorithm with full data of the validation

In [44]:
X_train = df_meta[["prophet", "prophet_multiva", "autoarima"]]
y_train = df_meta [["real_values"]]
lr.fit(X_train, y_train)

LinearRegression()

In [45]:
lr_predictions = lr.predict(testing_X)

In [46]:
df_final_predictions["lr_predictions"] = lr_predictions

In [47]:
print('Coefficients:', lr.coef_)
print('Intercept:', lr.intercept_)

Coefficients: [[8.6365826  0.39039091 8.07040303]]
Intercept: [-41.78173911]


In [48]:
for e in df_final_predictions.columns[1:]:
    mape=calculate_mape(df_test["y"],df_final_predictions[e])
    print(f'MAPE of {e}: {mape:.2f}%')

MAPE of prophet: 21.35%
MAPE of prophet_multiva: 21.87%
MAPE of autoarima: 21.07%
MAPE of average_predictions: 17.84%
MAPE of lr_predictions: 289.60%


In [50]:
df_final_predictions.head()

,ds,prophet,prophet_multiva,autoarima,average_predictions,lr_predictions
128,2022-06-20,2.643313,2.753302,2.579606,2.658740,2.940777
129,2022-06-27,2.670553,2.420241,2.613922,2.568239,3.322950
130,2022-07-04,2.720760,2.702383,2.644592,2.689245,4.114238
131,2022-07-11,2.700996,3.220187,2.672004,2.864396,4.366915
132,2022-07-18,2.586916,3.249357,2.696504,2.844259,3.590764


In [90]:
df_final_predictions

,ds,prophet,prophet_multiva,autoarima,average_predictions,lr_predictions,lr_fixed,lr_fixed_bigger0
128,2022-06-20,2.643313,2.753302,2.579606,2.658740,2.940777,2.509745,2.599155
129,2022-06-27,2.670553,2.420241,2.613922,2.568239,3.322950,2.603059,2.592123
130,2022-07-04,2.720760,2.702383,2.644592,2.689245,4.114238,2.583129,2.651096
131,2022-07-11,2.700996,3.220187,2.672004,2.864396,4.366915,2.571153,2.733701
132,2022-07-18,2.586916,3.249357,2.696504,2.844259,3.590764,2.691313,2.758726
133,2022-07-25,2.456163,2.668714,2.718401,2.614426,2.411554,2.908035,2.712809
134,2022-08-01,2.413950,2.820286,2.737972,2.657403,2.264093,2.951130,2.747236
135,2022-08-08,2.502929,3.168919,2.755464,2.809104,3.309835,2.870176,2.801997
136,2022-08-15,2.692913,3.286053,2.771098,2.916688,5.122545,2.749652,2.829054
137,2022-08-22,2.940951,2.914187,2.785070,2.880069,7.232340,2.657689,2.799602


# 8. Testing limited linear regression

In [52]:
from scipy.optimize import minimize

In [53]:
# Objective function to minimize (Mean Squared Error)
def objective(coefs, X, y):
    return np.mean((np.dot(X, coefs) - y) ** 2)



In [54]:
# Constraint that coefficients must sum to 1
constraints = {'type': 'eq', 'fun': lambda coefs: np.sum(coefs) - 1}

# Initial guess for the coefficients
initial_coefs = np.ones(X_train.shape[1]) / X_train.shape[1]



In [58]:
# Perform the optimization
result = minimize(objective, initial_coefs, args=(X_train.values, y_train.values), constraints=constraints)

# The optimized coefficients
coefs = result.x



In [59]:
print("Optimized coefficients:", coefs)
print("Sum of coefficients:", np.sum(coefs))

Optimized coefficients: [-0.69524429 -0.1472031   1.84244738]
Sum of coefficients: 1.0


In [63]:
coefs

array([-0.69524429, -0.1472031 ,  1.84244738])

In [70]:
lr_fixed = (testing_X['prophet'] * coefs[0] + testing_X['prophet_multiva'] * coefs[1] + testing_X['autoarima'] * coefs[2]).values

In [71]:
lr_fixed

array([2.50974455, 2.60305946, 2.58312939, 2.57115338, 2.69131305,
       2.90803455, 2.95112976, 2.87017566, 2.74965205, 2.65768873,
       2.41879218, 2.18740803, 1.85738528, 1.64117019, 1.56241691,
       1.5563166 , 1.66921951, 1.7159701 , 1.79908651, 1.82084264,
       1.83188565, 1.87928796, 1.87873535, 1.9259453 , 2.15163429,
       2.33319075, 2.38209907, 2.28705408, 2.10920276, 2.01126556,
       1.88934592, 1.98143336, 2.06409736, 2.10261395, 2.13526266,
       1.9898361 , 1.81309268, 1.84043028, 1.84521449, 1.89271203,
       1.98108377, 2.05326707, 2.12168266, 2.32216374, 2.55690406,
       2.78254482, 2.97350676, 3.02509627, 2.92265561, 2.89336573,
       2.78644208, 2.74919889, 2.7985902 , 2.78818142, 2.69640166])

In [72]:
df_final_predictions["lr_fixed"] = lr_fixed

In [74]:
for e in df_final_predictions.columns[1:]:
    mape=calculate_mape(df_test["y"],df_final_predictions[e])
    print(f'MAPE of {e}: {mape:.2f}%')

MAPE of prophet: 21.35%
MAPE of prophet_multiva: 21.87%
MAPE of autoarima: 21.07%
MAPE of average_predictions: 17.84%
MAPE of lr_predictions: 289.60%
MAPE of lr_fixed: 35.01%


# 9. Coefficients bigger than 0

In [77]:
from scipy.optimize import minimize

In [78]:
# Objective function to minimize (Mean Squared Error)
def objective(coefs, X, y):
    return np.mean((np.dot(X, coefs) - y) ** 2)



In [79]:
# Constraint that coefficients must sum to 1
constraints = {'type': 'eq', 'fun': lambda coefs: np.sum(coefs) - 1}

bounds = [(0, None) for _ in range(X_train.shape[1])]

# Initial guess for the coefficients
initial_coefs = np.ones(X_train.shape[1]) / X_train.shape[1]



In [80]:
# Perform the optimization
result = minimize(objective, initial_coefs, args=(X_train.values, y_train.values), constraints=constraints, bounds=bounds)

# The optimized coefficients
coefs_2 = result.x



In [81]:
print("Optimized coefficients:", coefs_2)
print("Sum of coefficients:", np.sum(coefs_2))

Optimized coefficients: [6.40987562e-17 1.12547416e-01 8.87452584e-01]
Sum of coefficients: 1.0


In [82]:
coefs_2

array([6.40987562e-17, 1.12547416e-01, 8.87452584e-01])

In [83]:
lr_fixed_bigger0 = (testing_X['prophet'] * coefs_2[0] + testing_X['prophet_multiva'] * coefs_2[1] + testing_X['autoarima'] * coefs_2[2]).values

In [84]:
lr_fixed_bigger0

array([2.59915476, 2.59212336, 2.65109618, 2.73370061, 2.7587261 ,
       2.71280906, 2.74723633, 2.80199729, 2.82905441, 2.79960206,
       2.85511692, 2.88126614, 2.98250906, 3.03514732, 3.04467763,
       3.05000956, 2.98737454, 2.98251124, 2.95920994, 2.99038316,
       3.01905909, 2.99687195, 3.01273986, 3.04536244, 3.01842173,
       3.0519078 , 3.11336068, 3.14453505, 3.12937149, 3.05124865,
       3.08591965, 3.06204551, 3.0794891 , 3.08102433, 3.00645979,
       3.02075935, 3.07182698, 3.02133022, 3.04472511, 3.06306317,
       3.04374908, 3.02016567, 3.00858462, 2.94731492, 2.92580463,
       2.93384278, 2.90998854, 2.87953941, 2.87527071, 2.80468505,
       2.85200143, 2.90462748, 2.89564388, 2.89583715, 2.93791385])

In [85]:
df_final_predictions["lr_fixed_bigger0"] = lr_fixed_bigger0

In [86]:
for e in df_final_predictions.columns[1:]:
    mape=calculate_mape(df_test["y"],df_final_predictions[e])
    print(f'MAPE of {e}: {mape:.2f}%')

MAPE of prophet: 21.35%
MAPE of prophet_multiva: 21.87%
MAPE of autoarima: 21.07%
MAPE of average_predictions: 17.84%
MAPE of lr_predictions: 289.60%
MAPE of lr_fixed: 35.01%
MAPE of lr_fixed_bigger0: 19.71%
